In [1]:
#Imports
import torch
import torchvision
import numpy as np
import torch.nn as nn
import torch.functional as F
import pandas as pd
import os
import torchvision.io as read_image
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
import matplotlib.pyplot as plt

In [37]:
# constructing the dataloader
class foodImageDataset(Dataset):
    def __init__(self, img_dir, data_dir, transformation = None):
        self.img_dir = img_dir
        self.ingr = pd.read_csv(data_dir, header=1,usecols=[5])
        self.transform = transformation
        self.labels = pd.read_csv(data_dir, header=1,usecols=[1,4])

    #Later augment the dataset to give more images per class to increase accuracy

    def __len__(self):
       return len(self.labels)
    
    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.labels[idx,1])
        print(img_path)
        image = read_image(img_path)
        label = self.labels[idx,0]
        if self.transform is not None:
            image = self.transform(image)
        return image, label

    def __getingrs__(self, index):
        return self.ingr[index]

In [19]:
#Model Declaration
class FoodModel(nn.Module):
    def __init__(self,img_size, output_size):
        super().__init__()
        self.conv1 = nn.Conv2d(3,55,kernel_size=(9,9), stride=3, padding=1)
        self.act1 = nn.ReLU()
        self.drop1 = nn.Dropout(0.3)
        print("1")
        self.conv2 = nn.Conv2d(55,55,kernel_size=(3,3), stride=1, padding=1)
        self.act2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(2,2))
        print("2")
        self.flat = nn.Flatten()
        print("3")
        self.fc3 = nn.Linear(int(55*55*55/8), int(output_size*2))
        self.act3 = nn.ReLU()
        self.drop3 = nn.Dropout(.5)
        print("4")
        self.fc4 = nn.Linear(output_size*2, output_size)
        print("5")
    def forward(self, image):
        #Transforming to Square
        image = transforms.Resize(169)
        print("1")
        #First layer
        image = self.act1(self.conv1(image))
        image = self.drop1(image)
        print("2")
        #Second layer
        image = self.act2(self.conv2(image))
        image = self.pool2(image)
        print("3")
        #Third layer
        image = self.flat(image)
        image = self.act3(self.fc3(image))
        image = self.drop3(image)
        print("4")
        #output
        image = self.fc4(image)
        return image
    
    def train(self, epochs, train_dl, loss_fn, opt):
        for epoch in range(epochs):
            count = 0
            for images, labels in train_dl:
                print(count)
                preds = self(images)
                loss = loss_fn(preds, labels)
                #training
                loss.backward()
                opt.step()
                opt.zero_grad()
                count += 1

In [5]:
#Constants
img_dir = r"C:\Users\Humperdink2\Documents\github\NewHacks2023\Food Images\Food Images"
data_dir = r"C:\Users\Humperdink2\Documents\github\NewHacks2023\Food Ingredients and Recipe Dataset with Image Name Mapping.csv"
batch_size = 128
learning_rate = 0.001
momentum = 0.9
img_size = 169
output_size = 13500

In [13]:
model = FoodModel(img_size, output_size)

1
2

4
5


In [14]:
opt = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [15]:
#Setting Model
loss_fn = nn.CrossEntropyLoss()


food = foodImageDataset(img_dir, data_dir, transformation=transforms.ToTensor())
food_dl = DataLoader(food, batch_size, shuffle=True)

In [38]:
#Running Model
model.train(5,food_dl,loss_fn,opt)

KeyError: (6957, 1)